In [1]:
import pandas as pd
import pickle
import ast
import collections
from pychord import Chord
import re
import xlsxwriter
from statistics import mean, median
import numpy as np
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk
#nltk.download('stopwords')
from collections import Counter
import spacy
from Levenshtein import distance as lev 
from fastDamerauLevenshtein import damerauLevenshtein as dlev
import string

### 1. Dados
Lendo e tratando os dados baixados

In [2]:
lista_dados_musica = []

for i in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']:
    with open('lista_dados_musica_'+ i + '.pkl', 'rb') as handle:
        for j in pickle.load(handle):
            lista_dados_musica.append(j)
    
df_dados_musicas = pd.DataFrame(lista_dados_musica, columns = ['artista', 'musica', 'tonalidade', 'cifra', 'letra']) 
df_dados_musicas.drop_duplicates(inplace=True)
df_dados_musicas.head()

,artista,musica,tonalidade,cifra,letra
0,disney,a-princesa-e-o-sapo-amigos-do-outro-lado,G,"[<b>Am</b>, <b>E7</b>, <b>Am</b>, <b>F</b>, <b...",[<p>Não me desrespeite homenzinho!<br/>Não faç...
1,disney,vencer-distancias,G,"[<b>D</b>, <b>E</b>, <b>A</b>, <b>D</b>, <b>E<...",[<p>Há um sonho em mim<br/>Vejo um lugar<br/>O...
2,disney,ciclo-sem-fim-o-rei-leao,Db,"[<b>Bb</b>, <b>Gm</b>, <b>Cm</b>, <b>F</b>, <b...",[<p>Desde o dia em que ao mundo chegamos<br/>C...
3,disney,belle-evermore,A,"[<b>A</b>, <b>A</b>, <b>D</b>, <b>Bm</b>, <b>F...",sem_info
4,disney,luca-il-gatto-e-la-volpe,A,"[<b>A</b>, <b>F#m</b>, <b>A</b>, <b>F#m</b>, <...",sem_info


In [3]:
len(df_dados_musicas)

32907

#### 1.1. Tratamento e limpeza

In [4]:
df_limpeza = df_dados_musicas.copy()

In [5]:
# Removendo artistas com alto índice de músicas em outros idiomas
df_limpeza = df_limpeza[~df_limpeza['artista'].isin(['disney',
'temas-infantis',
'steven-universo',
'o-rei-leao',
'frozen',
'bob-esponja',
'edu-falaschi',
'hermetica',
'bill-withers',
'luis-miguel',
'castro-cristian',
'leo-dan',
'sangtrait',
'paco-bandeira',
'marante'])]

In [6]:
df_limpeza = df_limpeza[(df_limpeza['cifra'] != 'sem_info')&(df_limpeza['cifra'] != '[]')].reset_index(drop=True)

In [7]:
def limpa_cifra(x):
    x_ = x.replace('[','').replace(']','').replace('<b>','').replace('</b>','').replace(' ','').split(',')
    return x_

def remover_pontuacao(texto):

    for character in string.punctuation:
        texto = texto.replace(character, '')
    return texto

def limpa_letra(x):
    if x == 'sem_info':
        x_ = 'sem_info'
    else:
        x_ = remover_pontuacao(unidecode(x.replace('[',' ').replace(']',' ').replace('<p>',' ').replace('</p>',' ').replace('<br/>', ' quebralinha '))).lower()
        x_ = x_.replace('spanspan', ' ').replace('classlrow', ' ').replace('2x', ' ').replace('3x', ' ').replace('4x', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ')
    return x_

df_limpeza['cifra_limpa'] = df_limpeza['cifra'].map(limpa_cifra)
df_limpeza['letra_limpa'] = df_limpeza['letra'].map(limpa_letra)


In [8]:
df_limpeza.head()

,artista,musica,tonalidade,cifra,letra,cifra_limpa,letra_limpa
0,3-palavrinhas,a-deus-dai-louvor,D,"[<b>E</b>, <b>F#</b>, <b>G</b>, <b>A7</b>, <b>...",[<p>A Deus dai louvor com todo ser<br/>Com tod...,"[E, F#, G, A7, A, G, F#, E, D, D, A7, A, G, F#...",a deus dai louvor com todo ser quebralinha co...
1,3-palavrinhas,pare,A,"[<b>A</b>, <b>D</b>, <b>A</b>, <b>D</b>, <b>E<...",[<p>Pare! Eu vou contar-lhe o que Cristo fez p...,"[A, D, A, D, E, A, D, E, A, D, E, A, D, E, A, ...",pare eu vou contarlhe o que cristo fez por mi...
2,3-palavrinhas,a-formiga,G,"[<b>G</b>, <b>D</b>, <b>G</b>, <b>D</b>, <b>G<...",[<p>A formiguinha corta folha e carrega<br/>Qu...,"[G, D, G, D, G, C, G, D, G, C, G, D, G, G, D, ...",a formiguinha corta folha e carrega quebralin...
3,3-palavrinhas,deus-criou-os-peixes,G,"[<b>G</b>, <b>Am</b>, <b>D</b>, <b>G</b>, <b>C...","[<p>Deus criou os peixes, para o rio e o mar<b...","[G, Am, D, G, C, Am, C, D, G, G, Am, D, G, C, ...",deus criou os peixes para o rio e o mar quebr...
4,3-palavrinhas,-feliz-o-lar,A,"[<b>A</b>, <b>E/G#</b>, <b>F#m</b>, <b>E</b>, ...",[<p>Se na família está Jesus<br/>É feliz o lar...,"[A, E/G#, F#m, E, Bm, E, A, A, E/G#, F#m, E, B...",se na familia esta jesus quebralinha e feliz ...


###### -> Eliminando acordes que só aparecem em 1 música

In [9]:
def cria_df_counter(lista):
    counter = collections.Counter(lista) 
    df_counter = pd.DataFrame.from_dict(dict(counter), orient='index').reset_index()
    df_counter.columns = ['acorde', 'qtde']
    df_counter['n_mus'] = df_counter['acorde'].apply(lambda y: df_limpeza['cifra_limpa'].apply(lambda x: y in x).sum())
    return df_counter

In [10]:
df_counter = cria_df_counter(df_limpeza['cifra_limpa'].sum())

In [11]:
df_counter.sort_values('qtde', ascending = False)

,acorde,qtde,n_mus
2,G,155122,14015
5,D,127020,12113
9,C,119288,11638
4,A,115654,10830
0,E,91962,8801
...,...,...,...
1723,G#7b13,1,1
4297,Am11/G,1,1
4296,Dm7(9)/E,1,1
3189,B7(b9/11),1,1


In [12]:
df_limpeza['cifra_limpa'] = df_limpeza['cifra_limpa'].apply(lambda x: [k for k in x if k not in list(df_counter[df_counter['n_mus'] == 1]['acorde'])] if (x!='sem_info') else x)


In [13]:
df_counter2 = cria_df_counter(df_limpeza['cifra_limpa'].sum())

In [14]:
len(df_counter2)

2771

In [15]:
df_counter2.to_excel('df_counter2.xlsx')

In [16]:
df_limpeza.head(2)

,artista,musica,tonalidade,cifra,letra,cifra_limpa,letra_limpa
0,3-palavrinhas,a-deus-dai-louvor,D,"[<b>E</b>, <b>F#</b>, <b>G</b>, <b>A7</b>, <b>...",[<p>A Deus dai louvor com todo ser<br/>Com tod...,"[E, F#, G, A7, A, G, F#, E, D, D, A7, A, G, F#...",a deus dai louvor com todo ser quebralinha co...
1,3-palavrinhas,pare,A,"[<b>A</b>, <b>D</b>, <b>A</b>, <b>D</b>, <b>E<...",[<p>Pare! Eu vou contar-lhe o que Cristo fez p...,"[A, D, A, D, E, A, D, E, A, D, E, A, D, E, A, ...",pare eu vou contarlhe o que cristo fez por mi...


In [17]:
# Harmonização das formas de escrita dos acordes

def ajuste_fino(acorde):
    if (acorde[-1] == '/')|((acorde[-1] == '-')&(len(acorde)==2))|((acorde[-1] == '+')&(len(acorde)==2)):
        acorde = acorde[:-1]
    elif ((acorde[0] == '-')&(len(acorde)==2))|((acorde[0] == '+')&(len(acorde)==2)):
        acorde = acorde[1:]
    return acorde

def ajuste_cifra(lista):
    if type(lista)==str:
        resultado = lista
    else:
        lista_ajustada = [k for k in lista if k not in ['C7-B7-Bb7-A7', 'EADGBE', 'C-B', 'C-G-A-F', 'G-D', 'A-C', 'CG', 'CC', 'CD', 'EB']]
        lista_ajustada = [k.replace('7M', '7+')\
                           .replace('Maj7', '7+')\
                           .replace('maj7', '7+')\
                           .replace('Maj9', '9+')\
                           .replace('maj9', '9+')\
                           .replace('dim', 'º')\
                           .replace('°', 'º')\
                           .replace('5#', '5+')\
                           .replace('m7(5-)', 'm7(b5)')\
                           .replace('m7/5-', 'm7(b5)')\
                           .replace('m5-/7', 'm7(b5)')\
                           .replace('m5-7', 'm7(b5)')\
                           .replace('m7/5b', 'm7(b5)')\
                           .replace('7m(b5)', '7(b5)')\
                           .replace('m7/b5', 'm7(b5)')\
                           .replace('m7(5b)', 'm7(b5)')\
                           .replace('m7b5', 'm7(b5)')\
                           .replace('/b13', '/13-')\
                           .replace('(13b)', '(13-)')\
                           .replace('(b13)', '(13-)')\
                           .replace('/13b', '/13-')\
                           .replace('(2x)', '')\
                           .replace('(2X)', '') for k in lista_ajustada]
        lista_ajustada = [k for k in lista_ajustada if k != '']
        lista_ajustada = [ajuste_fino(k) for k in lista_ajustada]
        resultado = lista_ajustada

    return resultado
            

In [18]:
df_limpeza['cifra_limpa'] = df_limpeza['cifra_limpa'].map(ajuste_cifra)

In [19]:
len(df_limpeza)

31397

In [20]:
df_counter3 = cria_df_counter(df_limpeza['cifra_limpa'].sum())
df_counter3.to_excel('df_counter3.xlsx')

In [21]:
len(df_counter3)

2476

In [22]:
df_limpeza.to_pickle('df_limpeza.pkl')

### 2. Análise
Criação dos indicadores de análise das cifras e letras

#### 2.1. Análise - indicadores básicos de cifras

###### -> Número de acordes, número de acordes distintos, complexidade do acorde, raridade do acorde, número e percentual de acordes pertencentes ao campo harmônico da tonalidade, tamanho médio da progressão de acordes dentro da música

In [23]:
df_limpeza = pd.read_pickle('df_limpeza.pkl')
df_analise_harmonia = df_limpeza.copy()

In [24]:
### Aqui, alguns ajustes necessários à criação das métricas

def reduz_radical(acorde): # reduz acorde ao radical
    if len(acorde) == 1:
        radical = acorde
    elif len(acorde) == 2:
        if acorde[1] in ['b','#', 'm', 'º']:
            radical = acorde
        else:
            radical = acorde[0]
    else:
        if '#m7(b5)' in acorde:
            radical = acorde[:8]
        elif 'bm7(b5)' in acorde:
            radical = acorde[:8]
        elif 'm7(b5)' in acorde:
            radical = acorde[:7]
        elif '#m(b5)' in acorde:
            radical = acorde[:7]
        elif 'bm(b5)' in acorde:
            radical = acorde[:7]
        elif 'm(b5)' in acorde:
            radical = acorde[:6]
        elif acorde[1:3] in ['bm', '#m', 'bº', '#º']:
            radical = acorde[:3]
        elif acorde[1] in ['b', '#', 'm', 'º']:
            radical = acorde[:2]
        else:
            radical = acorde[0]

    return radical

def grau_1(acorde, ton): # verifica se acorde é o primeiro grau do campo harmônico
    if len(acorde)==len(ton):
        if acorde==ton:
            teste = True
        else:
            teste = False
            
    elif len(acorde)>len(ton):
        if ton[-1]=='m':
            if (acorde[:len(ton)]==ton)&(acorde[len(ton)]!='#')&(acorde[len(ton)]!='b'):
                teste = True
            else:
                teste = False
        else:
            if (acorde[:len(ton)]==ton)&(acorde[len(ton)]!='#')&(acorde[len(ton)]!='b')&(acorde[len(ton)]!='m'):
                teste = True
            else:
                teste = False
    else:
        teste = False
    return teste

def identifica_grau_1(row): # verifica se acorde é o primeiro grau do campo harmônico
    ton = row['tonalidade']
    lista_teste = [grau_1(k, ton) for k in row['cifra_limpa_reduzida']]
    return lista_teste

#Acordes no contexto dos campos harmônicos
dict_campo_harmonico_simples = {
'C': ['C', 'Dm', 'Em', 'F', 'G', 'Am', 'Bm(b5)', 'Bm7(b5)', 'Bº'],
'G': ['G', 'Am', 'Bm', 'C', 'D', 'Em', 'F#m(b5)', 'F#m7(b5)', 'F#º'],
'D': ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#m(b5)', 'C#m7(b5)', 'C#º'],
'A': ['A', 'Bm', 'C#m', 'D', 'E', 'F#m', 'G#m(b5)', 'G#m7(b5)', 'G#º'],
'E': ['E', 'F#m', 'G#m', 'A', 'B', 'C#m', 'D#m(b5)', 'D#m7(b5)', 'D#º'],
'B': ['B', 'C#m', 'D#m', 'E', 'F#', 'G#m', 'A#m(b5)', 'A#m7(b5)', 'A#º'],
'F#': ['F#', 'G#m', 'A#m', 'B', 'C#', 'D#m', 'E#m(b5)', 'E#m7(b5)', 'E#º', 'Gb', 'Abm', 'Bbm', 'Cb', 'Db', 'Ebm', 'Fm(b5)', 'Fm7(b5)', 'Fº'],
'C#': ['C#', 'D#m', 'E#m', 'F#', 'G#', 'A#m', 'B#m(b5)', 'B#m7(b5)', 'B#º', 'Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cm(b5)', 'Cm7(b5)', 'Cº'],
'F': ['F', 'Gm', 'Am', 'Bb', 'C', 'Dm', 'Em(b5)', 'Em7(b5)', 'Eº'],
'Bb': ['Bb', 'Cm', 'Dm', 'Eb', 'F', 'Gm', 'Am(b5)', 'Am7(b5)', 'Aº', 'A#', 'B#m', 'C##m', 'D#', 'E#', 'F##m', 'G##m(b5)', 'G##m7(b5)', 'G##º'],
'A#': ['Bb', 'Cm', 'Dm', 'Eb', 'F', 'Gm', 'Am(b5)', 'Am7(b5)', 'Aº', 'A#', 'B#m', 'C##m', 'D#', 'E#', 'F##m', 'G##m(b5)', 'G##m7(b5)', 'G##º'],
'Db': ['C#', 'D#m', 'E#m', 'F#', 'G#', 'A#m', 'B#m(b5)', 'B#m7(b5)', 'B#º', 'Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cm(b5)', 'Cm7(b5)', 'Cº'],
'Eb': ['Eb', 'Fm', 'Gm', 'Ab', 'Bb', 'Cm', 'Dm(b5)', 'Dm7(b5)', 'Dº', 'D#', 'E#m', 'F##m', 'G#', 'A#', 'B#m', 'C##m(b5)', 'C##m7(b5)', 'C##º'],
'D#': ['Eb', 'Fm', 'Gm', 'Ab', 'Bb', 'Cm', 'Dm(b5)', 'Dm7(b5)', 'Dº', 'D#', 'E#m', 'F##m', 'G#', 'A#', 'B#m', 'C##m(b5)', 'C##m7(b5)', 'C##º'],
'Ab': ['Ab', 'Bbm', 'Cm', 'Db', 'Eb', 'Fm', 'Gm(b5)', 'Gm7(b5)', 'Gº', 'G#', 'A#m', 'B#m', 'C#', 'D#', 'E#m', 'F##m(b5)', 'F##m7(b5)', 'F##º'],
'G#': ['Ab', 'Bbm', 'Cm', 'Db', 'Eb', 'Fm', 'Gm(b5)', 'Gm7(b5)', 'Gº', 'G#', 'A#m', 'B#m', 'C#', 'D#', 'E#m', 'F##m(b5)', 'F##m7(b5)', 'F##º'],
'Am': ['C', 'Dm', 'Em', 'F', 'G', 'Am', 'Bm(b5)', 'Bm7(b5)', 'Bº'],
'Em': ['G', 'Am', 'Bm', 'C', 'D', 'Em', 'F#m(b5)', 'F#m7(b5)', 'F#º'],
'Bm': ['D', 'Em', 'F#m', 'G', 'A', 'Bm', 'C#m(b5)', 'C#m7(b5)', 'C#º'],
'F#m': ['A', 'Bm', 'C#m', 'D', 'E', 'F#m', 'G#m(b5)', 'G#m7(b5)', 'G#º', 'Bbb', 'Cbm', 'Dbm', 'Ebb', 'Fb', 'Gbm', 'Abm(b5)', 'Abm7(b5)', 'Abº'],
'C#m': ['E', 'F#m', 'G#m', 'A', 'B', 'C#m', 'D#m(b5)', 'D#m7(b5)', 'D#º', 'Fb', 'Gbm', 'Abm', 'Bbb', 'Cb', 'Dbm', 'Ebm(b5)', 'Ebm7(b5)', 'Ebº'],
'G#m': ['B', 'C#m', 'D#m', 'E', 'F#', 'G#m', 'A#m(b5)', 'A#m7(b5)', 'A#º', 'Cb', 'Dbm', 'Ebm', 'Fb', 'Gb', 'Abm', 'Gbm(b5)', 'Gbm7(b5)', 'Gbº'],
'D#m': ['F#', 'G#m', 'A#m', 'B', 'C#', 'D#m', 'E#m(b5)', 'E#m7(b5)', 'E#º', 'Gb', 'Abm', 'Bbm', 'Cb', 'Db', 'Ebm', 'Fm(b5)', 'Fm7(b5)', 'Fº'],
'A#m': ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cm(b5)', 'Cm7(b5)', 'Cº', 'C#', 'D#m', 'E#m', 'F#', 'G#', 'A#m', 'B#m(b5)', 'B#m7(b5)', 'B#º'],
'Bbm': ['Db', 'Ebm', 'Fm', 'Gb', 'Ab', 'Bbm', 'Cm(b5)', 'Cm7(b5)', 'Cº', 'C#', 'D#m', 'E#m', 'F#', 'G#', 'A#m', 'B#m(b5)', 'B#m7(b5)', 'B#º'],
'Ebm': ['F#', 'G#m', 'A#m', 'B', 'C#', 'D#m', 'E#m(b5)', 'E#m7(b5)', 'E#º', 'Gb', 'Abm', 'Bbm', 'Cb', 'Db', 'Ebm', 'Fm(b5)', 'Fm7(b5)', 'Fº'],
'Dm': ['F', 'Gm', 'Am', 'Bb', 'C', 'Dm', 'Em(b5)', 'Em7(b5)', 'Eº'],
'Gm': ['Bb', 'Cm', 'Dm', 'Eb', 'F', 'Gm', 'Am(b5)', 'Am7(b5)', 'Aº'],
'Cm': ['Eb', 'Fm', 'Gm', 'Ab', 'Bb', 'Cm', 'Dm(b5)', 'Dm7(b5)', 'Dº'],
'Fm': ['Ab', 'Bbm', 'Cm', 'Db', 'Eb', 'Fm', 'Gm(b5)', 'Gm7(b5)', 'Gº']
}

def pertence_campo(row): # verifica se acorde está no escopo do campo harmônico da tonalidade
    ton = row['tonalidade']
    if ton == 'sem_info':
        lista_teste = 'sem_info'
    else:
        lista_ref_campo = dict_campo_harmonico_simples[ton]
        lista_teste = [k in lista_ref_campo for k in row['cifra_limpa_reduzida']]
    return lista_teste

def quantifica_complexidade(acorde): # verifica se acorde tem mais de 3 notas
    lista_elementos = ['2', '4', '6', '7', '9', '11', '13', 'º']
    res = 3 + len([k for k in lista_elementos if(k in acorde)])
    return res

def quantifica_raridade(acorde): # quantifica a raridade do acorde no universo de músicas (quanto mais usado em outras músicas menos valor)
    raridade = 1/df_counter3[df_counter3['acorde']==acorde].iloc[0]['qtde']
    return raridade

In [25]:
df_counter3 = pd.read_excel('df_counter3.xlsx')

In [26]:
df_analise_harmonia['cifra_limpa_reduzida'] = df_analise_harmonia['cifra_limpa'].apply(lambda x: [reduz_radical(k) for k in x])
df_analise_harmonia['lista_teste_grau_1'] = df_analise_harmonia.apply(identifica_grau_1, axis = 1)
df_analise_harmonia['lista_teste_pertence_campo'] = df_analise_harmonia.apply(pertence_campo, axis = 1)
df_analise_harmonia['cifra_limpa_distintos'] = df_analise_harmonia['cifra_limpa'].apply(lambda x: list(set(x)))
df_analise_harmonia['lista_complexidade_acorde'] = df_analise_harmonia['cifra_limpa'].apply(lambda x: [quantifica_complexidade(k) for k in x])
df_analise_harmonia['lista_raridade_acorde'] = df_analise_harmonia['cifra_limpa'].apply(lambda x: [quantifica_raridade(k) for k in x])


In [ ]:
df_analise_harmonia['num_acordes'] = df_analise_harmonia['cifra_limpa'].apply(lambda x: len(x))
df_analise_harmonia['num_acordes_distintos'] = df_analise_harmonia['cifra_limpa_distintos'].apply(lambda x: len(x))
df_analise_harmonia['num_acordes_fora_campo'] = df_analise_harmonia['lista_teste_pertence_campo'].apply(lambda x: x.count(False) if x!='sem_info' else 'sem_info')


In [ ]:
def perc_acordes(row):
    if row['num_acordes_fora_campo'] == 'sem_info':
        perc = 'sem_info'
    else:
        perc = row['num_acordes_fora_campo']/row['num_acordes']
    return perc


In [ ]:
df_analise_harmonia = df_analise_harmonia[df_analise_harmonia['num_acordes']>0]
df_analise_harmonia['perc_acordes_fora_campo'] = df_analise_harmonia.apply(perc_acordes, axis=1)


In [ ]:
def tamanho_medio_progressao(row):
    if row['lista_teste_grau_1'].count(True) == 0:
        tmp = row['num_acordes']
    else:
        tmp = row['num_acordes']/row['lista_teste_grau_1'].count(True)
    return tmp

In [ ]:
df_analise_harmonia['tamanho_medio_progressao'] = df_analise_harmonia.apply(tamanho_medio_progressao,axis=1)
df_analise_harmonia['media_complexidade'] = df_analise_harmonia['lista_complexidade_acorde'].apply(lambda x: mean(x))
df_analise_harmonia['media_raridade'] = df_analise_harmonia['lista_raridade_acorde'].apply(lambda x: mean(x))


In [25]:

def ident_decil(num, coluna):
    s = coluna
    s = s[s!='sem_info']
    decil = list(s.quantile([.1,.2,.3,.4,.5,.6,.7,.8,.9]))
    if num == 'sem_info':
        y = 'sem_info'
    elif num <= decil[0]:
        y = 1
    elif num <= decil[1]:
        y = 2
    elif num <= decil[2]:
        y = 3
    elif num <= decil[3]:
        y = 4
    elif num <= decil[4]:
        y = 5
    elif num <= decil[5]:
        y = 6
    elif num <= decil[6]:
        y = 7
    elif num <= decil[7]:
        y = 8
    elif num <= decil[8]:
        y = 9
    else:
        y = 10
    return y

In [27]:


for coluna in [df_analise_harmonia['num_acordes_distintos'], df_analise_harmonia['perc_acordes_fora_campo'], df_analise_harmonia['tamanho_medio_progressao'], df_analise_harmonia['media_raridade'], df_analise_harmonia['media_complexidade']]:
    df_analise_harmonia[coluna.name+'_decil'] = coluna.apply(lambda x: ident_decil(x, coluna))
    print(coluna.name)
    

num_acordes_distintos
perc_acordes_fora_campo
tamanho_medio_progressao
media_raridade
media_complexidade


###### -> Nessa última etapa, foram criados indicadores padronizados para cada indicador original, comparando-se o indicador de cada música com todas as demais, sendo as músicas então ranqueadas e divididas em 10 grupos (decis). Pertencer ao decil 1, por exemplo, significa que a música está entre as 10% com maior valor daquele indicador.

### ------------------------------------------- AJUSTE CAPO--------------------------------------------
Preciso identificar músicas que estão cifradas com uso de capotraste, pois a forma com que são apresentadas distorcem os dois indicadores relacionados ao campo harmônico.

In [ ]:
df_ajuste = df_analise_harmonia[df_analise_harmonia['perc_acordes_fora_campo'] != 'sem_info']
df_ajuste = df_ajuste[df_ajuste['perc_acordes_fora_campo']>.5][['artista', 'musica']]

In [ ]:
import requests
from bs4 import BeautifulSoup
import html5lib

#lista_teste_capo = []

count=4100
for artista in list(set(df_ajuste['artista']))[604:]:
    lista_musicas = list(df_ajuste[df_ajuste['artista']==artista]['musica'])
    for musica in lista_musicas:
        url = 'https://www.cifraclub.com/' + artista + '/' + musica
        page = requests.get(url)
        html_doc = page.text
        soup = BeautifulSoup(html_doc, 'html5lib')
        if soup.find(title="alterar capotraste") == None:
            ind_capo = 0
        else:
            ind_capo = 1
        lista_teste_capo.append([artista, musica, ind_capo])
        count+=1
        if count%50==0:
            print(artista, musica, count)

In [ ]:
with open('lista_teste_capo.pkl', 'wb') as f:
    pickle.dump(lista_teste_capo, f)

In [28]:
with open('lista_teste_capo.pkl', 'rb') as handle:
    lista_teste_capo = pickle.load(handle)

In [29]:
lista_capo = [k for k in lista_teste_capo if k[2]==1]
len(lista_capo)

1161

In [30]:
def marca_casos_capo(row):
    if list(row[['artista', 'musica']]) in [[k[0], k[1]] for k in lista_capo]:
        res = 'capo'
    else:
        res = 'ok'
    return res

df_analise_harmonia['capo'] = df_analise_harmonia.apply(marca_casos_capo, axis=1)

df_analise_capo = df_analise_harmonia[df_analise_harmonia['capo']=='capo'].reset_index(drop=True)
df_analise_ok = df_analise_harmonia[df_analise_harmonia['capo']=='ok'].reset_index(drop=True)
df_analise_capo['num_acordes_fora_campo'] = 'sem_info'
df_analise_capo['perc_acordes_fora_campo'] = 'sem_info'
df_analise_capo['tamanho_medio_progressao'] = 'sem_info'
df_analise_harmonia = pd.concat([df_analise_capo, df_analise_ok]).reset_index(drop=True)
del(df_analise_harmonia['capo'])

###### -> Após esse ajuste, os indicadores padronizados são recalculados, desprezando-se os casos marcados

In [31]:
for coluna in [df_analise_harmonia['perc_acordes_fora_campo'], df_analise_harmonia['tamanho_medio_progressao']]:
    df_analise_harmonia[coluna.name+'_decil'] = coluna.apply(lambda x: ident_decil(x, coluna))
    print(coluna.name)

perc_acordes_fora_campo
tamanho_medio_progressao


### ----------------------------------------------x--------------------------------------------

In [99]:
df_analise_harmonia.sample(5)

,artista,musica,tonalidade,cifra,letra,cifra_limpa,letra_limpa,cifra_limpa_reduzida,lista_teste_grau_1,lista_teste_pertence_campo,...,num_acordes_fora_campo,perc_acordes_fora_campo,tamanho_medio_progressao,media_complexidade,media_raridade,num_acordes_distintos_decil,perc_acordes_fora_campo_decil,tamanho_medio_progressao_decil,media_raridade_decil,media_complexidade_decil
16388,reginaldo-rossi,dama-de-vermelho,C,"[<b>Dm</b>, <b>G</b>, <b>C</b>, <b>Am</b>, <b>...","[<p>Garçom, olhe pelo espelho<br/>A dama de ve...","[Dm, G, C, Am, Dm, G, C, G7, C, Em, F, G, F, G...",garcom olhe pelo espelho quebralinha a dama d...,"[Dm, G, C, Am, Dm, G, C, G, C, Em, F, G, F, G,...","[False, False, True, False, False, False, True...","[True, True, True, True, True, True, True, Tru...",...,0,0.0,3.84,3.062500,0.000015,5,1,4,2,4
7833,zezo,fortaleza,Eb,"[<b>Cm</b>, <b>G7</b>, <b>Cm</b>, <b>C7</b>, <...",[<p>Fortaleza se eu pudesse<br/>Se papai do cé...,"[Cm, G7, Cm, C7, Fm, Cm, G7, C, G7, C, Fm, Cm,...",fortaleza se eu pudesse quebralinha se papai ...,"[Cm, G, Cm, C, Fm, Cm, G, C, G, C, Fm, Cm, G, Cm]","[False, False, False, False, False, False, Fal...","[True, False, True, False, True, True, False, ...",...,7,0.5,14.0,3.357143,0.000051,3,9,8,5,7
14491,marcelo-camelo,meu-amor-e-teu,G,"[<b>G</b>, <b>Am</b>, <b>Am7</b>, <b>D</b>, <b...",[<p>Meu amor é teu<br/>Mas dou-te mais uma vez...,"[G, Am, Am7, D, D7, G, G, Am, Am7, D, D7, G, C...",meu amor e teu quebralinha mas doute mais uma...,"[G, Am, Am, D, D, G, G, Am, Am, D, D, G, C, G,...","[True, False, False, False, False, True, True,...","[True, True, True, True, True, True, True, Tru...",...,0,0.0,3.3125,3.245283,0.000017,4,1,3,3,6
11748,gal-costa,estrela-de-papel,A,"[<b>C</b>, <b>Bb/C</b>, <b>G#/C</b>, <b>Eb/Bb<...",[<p>Nas imagens de um gibi<br/>Ele é um herói<...,"[C, Bb/C, G#/C, Eb/Bb, F/C, C, C, Am, Dm, Dm/C...",nas imagens de um gibi quebralinha ele e um h...,"[C, Bb, G#, Eb, F, C, C, Am, Dm, Dm, G, F, G, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",...,55,0.733333,15.0,3.080000,0.001185,10,10,9,9,5
13976,ivan-lins,vitoriosa,D,"[<b>D7M</b>, <b>A7(4/9)</b>, <b>D7M</b>, <b>C7...",[<p>Quero sua risada mais gostosa<br/>Esse seu...,"[D7+, A7(4/9), D7+, C7(4/9), C7(9), Bm7(9), D7...",quero sua risada mais gostosa quebralinha ess...,"[D, A, D, C, C, Bm, D, Em, A, D, C, C, D, A, D...","[True, False, True, False, False, False, True,...","[True, True, True, False, False, True, True, T...",...,26,0.282609,4.842105,4.695652,0.002394,10,7,6,9,10


In [33]:
df_analise_harmonia.to_pickle('df_analise_harmonia.pkl')

###### -> Agregando indicadores por artista, pela mediana das músicas. Além dos indicadores das músicas agregados pela mediana, foram incluídos mais 2 indicadores do artista: a quantidade de músicas e a quantidade geral de acordes diferentes utilizados

In [34]:
df_artista = df_analise_harmonia[['artista', 'num_acordes_distintos_decil', 'perc_acordes_fora_campo_decil', 'tamanho_medio_progressao_decil', 'media_raridade_decil', 'media_complexidade_decil']].replace('sem_info', np.nan)
df_artista = df_artista.groupby(['artista']).median().reset_index()


In [35]:
with open('dict_musicas_por_artista.pkl', 'rb') as handle:
    dict_musicas_por_artista = pickle.load(handle)

In [36]:
# Agregando indicadores gerais do artista
lista_qtde = []
for i in list(dict_musicas_por_artista.keys()):
    lista_qtde.append([i, len(dict_musicas_por_artista[i])])
df_qtde_cifras = pd.DataFrame(lista_qtde)
    
df_artista = df_artista.merge(df_qtde_cifras, how = 'left', left_on='artista', right_on=0)
del(df_artista[0])
df_artista['qtde_musicas_decil'] = df_artista[1].apply(lambda x: ident_decil(x, df_artista[1]))
del(df_artista[1])
    


In [37]:
lista_geral_qtde_acordes = []
for i in list(df_artista['artista']):
    lista_geral = list(set(df_analise_harmonia[df_analise_harmonia['artista'] == i]['cifra_limpa'].sum()))
    tam_lista = len(lista_geral)
    lista_geral_qtde_acordes.append([i, tam_lista])
df_qtde_acordes = pd.DataFrame(lista_geral_qtde_acordes)

df_artista = df_artista.merge(df_qtde_acordes, how = 'left', left_on='artista', right_on=0)
df_artista['qtde_geral_acordes_decil'] = df_artista[1].apply(lambda x: ident_decil(x, df_artista[1]))
del(df_artista[0])
del(df_artista[1])



In [38]:
df_qtde_cifras.to_pickle('df_qtde_cifras.pkl')
df_qtde_acordes.to_pickle('df_qtde_acordes.pkl')

###### -> Calculando nota geral da complexidade harmônica, por artista

In [39]:
df_artista['nota_harmonia'] = df_artista[list(df_artista)[1:]].sum(axis=1)/7

In [40]:
df_artista.sort_values('nota_harmonia', ascending=False).head(20)

,artista,num_acordes_distintos_decil,perc_acordes_fora_campo_decil,tamanho_medio_progressao_decil,media_raridade_decil,media_complexidade_decil,qtde_musicas_decil,qtde_geral_acordes_decil,nota_harmonia
151,chico-buarque,10.0,9.0,8.0,10.0,10.0,10,10,9.571429
349,ivan-lins,10.0,9.0,7.0,10.0,10.0,10,10,9.428571
237,emilio-santiago,10.0,9.0,8.0,10.0,10.0,9,10,9.428571
367,joao-gilberto,10.0,9.0,8.0,10.0,10.0,9,10,9.428571
782,tom-jobim,10.0,9.0,8.0,10.0,10.0,9,10,9.428571
211,djavan,9.0,9.0,8.0,10.0,10.0,10,10,9.428571
364,joao-bosco,10.0,9.0,8.0,10.0,10.0,9,10,9.428571
382,jorge-vercillo,9.0,8.0,8.0,10.0,10.0,10,10,9.285714
788,toquinho,10.0,9.0,7.0,9.0,9.0,10,10,9.142857
235,elis-regina,9.0,9.0,8.0,9.0,9.5,9,10,9.071429


In [41]:
df_artista.to_pickle('df_artista.pkl')

#### 2.2. Análise - indicadores de letras

###### -> Número de palavras da letra, raridade das palavras, número de rimas, número de rimas envolvendo classes gramaticais diferentes

In [205]:
df_analise_letra = pd.read_pickle('df_limpeza.pkl')

In [206]:
df_analise_letra = df_analise_letra[df_analise_letra['letra_limpa'] != 'sem_info'][['artista', 'musica', 'letra', 'letra_limpa']].reset_index(drop=True)
df_analise_letra = df_analise_letra[df_analise_letra['letra_limpa'].apply(lambda x: len(x))>50]

In [207]:
len(df_analise_letra)

26632

In [208]:
df_analise_letra['letra_limpa_lista'] = df_analise_letra['letra_limpa'].apply(lambda x: [k for k in x.split(' ') if k not in [' ','']])


In [209]:
df_analise_letra.head()

,artista,musica,letra,letra_limpa,letra_limpa_lista
0,3-palavrinhas,a-deus-dai-louvor,[<p>A Deus dai louvor com todo ser<br/>Com tod...,a deus dai louvor com todo ser quebralinha co...,"[a, deus, dai, louvor, com, todo, ser, quebral..."
1,3-palavrinhas,pare,[<p>Pare! Eu vou contar-lhe o que Cristo fez p...,pare eu vou contarlhe o que cristo fez por mi...,"[pare, eu, vou, contarlhe, o, que, cristo, fez..."
2,3-palavrinhas,a-formiga,[<p>A formiguinha corta folha e carrega<br/>Qu...,a formiguinha corta folha e carrega quebralin...,"[a, formiguinha, corta, folha, e, carrega, que..."
3,3-palavrinhas,deus-criou-os-peixes,"[<p>Deus criou os peixes, para o rio e o mar<b...",deus criou os peixes para o rio e o mar quebr...,"[deus, criou, os, peixes, para, o, rio, e, o, ..."
4,3-palavrinhas,-feliz-o-lar,[<p>Se na família está Jesus<br/>É feliz o lar...,se na familia esta jesus quebralinha e feliz ...,"[se, na, familia, esta, jesus, quebralinha, e,..."


In [210]:
df_analise_letra['num_palavras_letra'] = df_analise_letra['letra_limpa_lista'].apply(lambda x: len([k for k in x if (k!= 'quebralinha')&(len(k)>1)]))
df_analise_letra['num_palavras_distintas_letra'] = df_analise_letra['letra_limpa_lista'].apply(lambda x: len(set([k for k in x if (k!= 'quebralinha')&(len(k)>1)])))
df_analise_letra['perc_palavras_distintas_letra'] = df_analise_letra['num_palavras_distintas_letra']/df_analise_letra['num_palavras_letra']


In [236]:
#raridade (método de cálculo da raridade pela contagem de palavras, alternativa ao tf-idf). 
# Acabei não utilizando o indicador decorrente desse processo, pois optei pelo tf-idf.

def cria_df_counter_letra(lista):
    counter = collections.Counter(lista) 
    df_counter = pd.DataFrame.from_dict(dict(counter), orient='index').reset_index()
    df_counter.columns = ['palavra', 'qtde']
    return df_counter

lista_palavras_total = df_analise_letra['letra_limpa_lista'].sum()
df_counter_palavras_total = cria_df_counter_letra(lista_palavras_total)


In [237]:
def raridade_media_letra(lista):
    df_temp = pd.DataFrame({0: [k for k in lista if k!='quebralinha']})
    df_temp = df_temp.merge(df_counter_palavras_total, left_on=0, right_on='palavra', how='inner')
    ind_raridade = (1/df_temp['qtde']).mean()
    return ind_raridade

df_analise_letra['raridade_media_letra'] = df_analise_letra['letra_limpa_lista'].apply(raridade_media_letra)


In [238]:
df_analise_letra['raridade_media_letra_ajustado'] = df_analise_letra['raridade_media_letra']*df_analise_letra['perc_palavras_distintas_letra']


In [213]:
stopwords = nltk.corpus.stopwords.words('portuguese') + ['quebralinha']

doc_corpus = df_analise_letra['letra_limpa'].to_list()
vec = TfidfVectorizer(stop_words=stopwords)
matrix=vec.fit_transform(doc_corpus)

In [214]:
print("Nome das variáveis n",vec.get_feature_names_out(),'\n')
print("Matrix n",matrix.shape,"n",matrix.toarray())

Nome das variáveis n ['00' '01' '013' ... 'zuzumba' 'zyz' 'zzz'] 

Matrix n (26632, 74508) n [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [215]:
df_analise_letra['tf_idf_sum'] = matrix.toarray().sum(axis=1)
df_analise_letra['tf_idf_medio'] = np.true_divide(matrix.sum(1),(matrix!=0).sum(1))

#tf_idf_medio_ajustado
df_analise_letra['tf_idf_medio_ajustado'] = df_analise_letra['tf_idf_medio']*df_analise_letra['perc_palavras_distintas_letra']


In [216]:
df_analise_letra.iloc[14705]

artista                                                                  tom-jobim
musica                                                                  insensatez
letra                            [<p>A insensatez que você fez<br/>Coração mais...
letra_limpa                       a insensatez que voce fez quebralinha coracao...
letra_limpa_lista                [a, insensatez, que, voce, fez, quebralinha, c...
num_palavras_letra                                                              69
num_palavras_distintas_letra                                                    45
perc_palavras_distintas_letra                                             0.652174
tf_idf_sum                                                                5.590412
tf_idf_medio                                                              0.155289
tf_idf_medio_ajustado                                                     0.101276
Name: 14710, dtype: object

In [217]:
nlp = spacy.load("pt_core_news_sm")
#list to store the tokens and pos tags 
token = []
pos = []
token_pos = []
 
for sent in nlp.pipe(df_analise_letra['letra_limpa']):
    if sent.has_annotation('DEP'):
        #add the tokens present in the sentence to the token list
        token.append([word.text for word in sent])
        #add the pos tage for each token to the pos list
        pos.append([word.pos_ for word in sent])
        token_pos.append([[word.text, word.pos_] for word in sent])

df_analise_letra['letra_tokens_morfossintaticos'] = token
df_analise_letra['letra_elementos_morfossintaticos'] = pos
df_analise_letra['letra_morfossintaxe'] = token_pos


In [218]:
df_analise_letra['num_elementos_morfossintaticos'] = df_analise_letra['letra_elementos_morfossintaticos'].apply(lambda x: len(set(x)))

def num_elementos_morfossintaticos_relativo(row):
    y = row['num_elementos_morfossintaticos']/len(row['letra_elementos_morfossintaticos'])
    return y
    
    
df_analise_letra['num_elementos_morfossintaticos_relativo'] = df_analise_letra.apply(num_elementos_morfossintaticos_relativo, axis=1)


In [219]:
# criando uma lista com todas as palavras das quais foram obtidos elementos morfossintáticos
# (servirá para busca de rimas, em caderno a parte)

def lista_alvos(lista):
    lista_alvos = []
    for i in range(len(lista)):
        if lista[i]=='quebralinha':
            lista_alvos.append(lista[i-1])
    return lista_alvos

lista_tokens = list(set([k for k in df_analise_letra['letra_tokens_morfossintaticos'].map(lista_alvos).sum() if len(k)>2]))
with open('lista_tokens.pkl', 'wb') as f:
    pickle.dump(lista_tokens, f)


In [220]:
with open('dict_rimas.pkl', 'rb') as handle:
    dict_rimas = pickle.load(handle)

In [221]:
df_analise_letra.head()

,artista,musica,letra,letra_limpa,letra_limpa_lista,num_palavras_letra,num_palavras_distintas_letra,perc_palavras_distintas_letra,tf_idf_sum,tf_idf_medio,tf_idf_medio_ajustado,letra_tokens_morfossintaticos,letra_elementos_morfossintaticos,letra_morfossintaxe,num_elementos_morfossintaticos,num_elementos_morfossintaticos_relativo
0,3-palavrinhas,a-deus-dai-louvor,[<p>A Deus dai louvor com todo ser<br/>Com tod...,a deus dai louvor com todo ser quebralinha co...,"[a, deus, dai, louvor, com, todo, ser, quebral...",187,20,0.106952,3.104270,0.206951,0.022134,"[ , a, deus, dai, louvor, com, todo, ser, queb...","[SPACE, DET, PROPN, ADP, NOUN, ADP, DET, AUX, ...","[[ , SPACE], [a, DET], [deus, PROPN], [dai, AD...",12,0.043636
1,3-palavrinhas,pare,[<p>Pare! Eu vou contar-lhe o que Cristo fez p...,pare eu vou contarlhe o que cristo fez por mi...,"[pare, eu, vou, contarlhe, o, que, cristo, fez...",59,19,0.322034,2.818517,0.234876,0.075638,"[ , pare, eu, vou, contarlhe, o, que, cristo, ...","[SPACE, NOUN, PRON, AUX, VERB, PRON, PRON, PRO...","[[ , SPACE], [pare, NOUN], [eu, PRON], [vou, A...",10,0.135135
2,3-palavrinhas,a-formiga,[<p>A formiguinha corta folha e carrega<br/>Qu...,a formiguinha corta folha e carrega quebralin...,"[a, formiguinha, corta, folha, e, carrega, que...",102,25,0.245098,3.332308,0.185128,0.045375,"[ , a, formiguinha, corta, folha, e, carrega, ...","[SPACE, DET, PROPN, VERB, PROPN, CCONJ, VERB, ...","[[ , SPACE], [a, DET], [formiguinha, PROPN], [...",12,0.089552
3,3-palavrinhas,deus-criou-os-peixes,"[<p>Deus criou os peixes, para o rio e o mar<b...",deus criou os peixes para o rio e o mar quebr...,"[deus, criou, os, peixes, para, o, rio, e, o, ...",105,27,0.257143,3.846807,0.183181,0.047104,"[ , deus, criou, os, peixes, para, o, rio, e, ...","[SPACE, VERB, VERB, DET, NOUN, ADP, DET, NOUN,...","[[ , SPACE], [deus, VERB], [criou, VERB], [os,...",13,0.089041
4,3-palavrinhas,-feliz-o-lar,[<p>Se na família está Jesus<br/>É feliz o lar...,se na familia esta jesus quebralinha e feliz ...,"[se, na, familia, esta, jesus, quebralinha, e,...",100,12,0.120000,1.895724,0.236966,0.028436,"[ , se, na, familia, esta, jesus, quebralinha,...","[SPACE, SCONJ, ADP, NOUN, DET, ADJ, X, CCONJ, ...","[[ , SPACE], [se, SCONJ], [na, ADP], [familia,...",11,0.064327


In [222]:
def termino(palavra): # identifica terminação da palavra, a ser usada na busca por rimas
    if len(palavra)<3:
        parte=palavra
    else:
        if palavra[-1]=='r':
            parte=palavra[-3:-1]
        else:
            parte=palavra[-3:]
    return parte

def lista_comparacoes(lista_de_listas): # cria lista de palavras para comparação
    lista_comparaveis = []
    lista_indices_marcador = []
    lista_alvos = []
    for i in range(len(lista_de_listas)):
        if lista_de_listas[i][0]=='quebralinha':
            lista_indices_marcador.append(i)
            lista_alvos.append(lista_de_listas[i-1])
    lista_alvos = lista_alvos[:-2]

    for i in range(len(lista_alvos)):
        lista_ = lista_de_listas[lista_indices_marcador[i]+1:lista_indices_marcador[i+2]]
        lista_ = [k for k in lista_ if k[0]!='quebralinha']
        for j in lista_:
            lista_comparaveis.append([lista_alvos[i], j])
    lista_comparaveis = [k for k in lista_comparaveis if (len(k[0][0])>1)&(len(k[1][0])>1)&(k[0][0]!='quebralinha')&(k[1][0]!='quebralinha')]
    lista_comparaveis = [k for k in lista_comparaveis if k[0][0]!=k[1][0]]
    return lista_comparaveis

def identifica_rima(lista_comparacoes): 
# calcula distância de Damerau Levenshtein para cada combinação e retorna as com indicador superior a 0.5
# além disso, pesquisa no dicionário de rimas

    lista_=[]
    for i in lista_comparacoes:
        if len(termino(i[1][0]))>2:
            dist_lev = dlev(termino(i[0][0]),termino(i[1][0]))
        else: 
            dist_lev = 0
        retorno_dict_rimas = 0
        if i[0][0] in list(dict_rimas.keys()):
            if i[1][0] in [k.split(' ')[-1] for k in dict_rimas[i[0][0]]]:
                retorno_dict_rimas = 1
        lista_.append([i[0], i[1], dist_lev, retorno_dict_rimas])
            
    lista_rimas = [k for k in lista_ if (k[2]>.5)|(k[3]==1)]
    return lista_rimas

In [223]:
df_analise_letra['lista_rimas'] = df_analise_letra['letra_morfossintaxe'].apply(lambda x: identifica_rima(lista_comparacoes(x)))


In [224]:
df_analise_letra['num_rimas'] = df_analise_letra['lista_rimas'].apply(lambda x: len(x))

def num_rimas_relativo(row):
    y = row['num_rimas']/row['num_palavras_letra']
    return y

df_analise_letra['num_rimas_relativo'] = df_analise_letra.apply(num_rimas_relativo, axis=1)

In [225]:
def num_rimas_classes_dif(lista_rimas):
    lista_alvo = [k for k in lista_rimas if k[0][1]!=k[1][1]]
    return len(lista_alvo)

df_analise_letra['num_rimas_classes_diferentes'] = df_analise_letra['lista_rimas'].map(num_rimas_classes_dif)

def num_rimas_classe_dif_relativo(row):
    y = row['num_rimas_classes_diferentes']/row['num_palavras_letra']
    return y

df_analise_letra['num_rimas_classes_dif_relativo'] = df_analise_letra.apply(num_rimas_classe_dif_relativo, axis=1)

In [226]:
df_analise_letra.head()

,artista,musica,letra,letra_limpa,letra_limpa_lista,num_palavras_letra,num_palavras_distintas_letra,perc_palavras_distintas_letra,tf_idf_sum,tf_idf_medio,...,letra_tokens_morfossintaticos,letra_elementos_morfossintaticos,letra_morfossintaxe,num_elementos_morfossintaticos,num_elementos_morfossintaticos_relativo,lista_rimas,num_rimas,num_rimas_relativo,num_rimas_classes_diferentes,num_rimas_classes_dif_relativo
0,3-palavrinhas,a-deus-dai-louvor,[<p>A Deus dai louvor com todo ser<br/>Com tod...,a deus dai louvor com todo ser quebralinha co...,"[a, deus, dai, louvor, com, todo, ser, quebral...",187,20,0.106952,3.104270,0.206951,...,"[ , a, deus, dai, louvor, com, todo, ser, queb...","[SPACE, DET, PROPN, ADP, NOUN, ADP, DET, AUX, ...","[[ , SPACE], [a, DET], [deus, PROPN], [dai, AD...",12,0.043636,"[[[ser, AUX], [use, VERB], 0.6666666666666666,...",12,0.064171,12,0.064171
1,3-palavrinhas,pare,[<p>Pare! Eu vou contar-lhe o que Cristo fez p...,pare eu vou contarlhe o que cristo fez por mi...,"[pare, eu, vou, contarlhe, o, que, cristo, fez...",59,19,0.322034,2.818517,0.234876,...,"[ , pare, eu, vou, contarlhe, o, que, cristo, ...","[SPACE, NOUN, PRON, AUX, VERB, PRON, PRON, PRO...","[[ , SPACE], [pare, NOUN], [eu, PRON], [vou, A...",10,0.135135,"[[[perdao, NOUN], [coracao, NOUN], 0.666666666...",2,0.033898,0,0.000000
2,3-palavrinhas,a-formiga,[<p>A formiguinha corta folha e carrega<br/>Qu...,a formiguinha corta folha e carrega quebralin...,"[a, formiguinha, corta, folha, e, carrega, que...",102,25,0.245098,3.332308,0.185128,...,"[ , a, formiguinha, corta, folha, e, carrega, ...","[SPACE, DET, PROPN, VERB, PROPN, CCONJ, VERB, ...","[[ , SPACE], [a, DET], [formiguinha, PROPN], [...",12,0.089552,"[[[carrega, VERB], [pega, ADJ], 1.0, 1], [[peg...",16,0.156863,14,0.137255
3,3-palavrinhas,deus-criou-os-peixes,"[<p>Deus criou os peixes, para o rio e o mar<b...",deus criou os peixes para o rio e o mar quebr...,"[deus, criou, os, peixes, para, o, rio, e, o, ...",105,27,0.257143,3.846807,0.183181,...,"[ , deus, criou, os, peixes, para, o, rio, e, ...","[SPACE, VERB, VERB, DET, NOUN, ADP, DET, NOUN,...","[[ , SPACE], [deus, VERB], [criou, VERB], [os,...",13,0.089041,"[[[nadar, VERB], [grandao, ADJ], 0.66666666666...",19,0.180952,15,0.142857
4,3-palavrinhas,-feliz-o-lar,[<p>Se na família está Jesus<br/>É feliz o lar...,se na familia esta jesus quebralinha e feliz ...,"[se, na, familia, esta, jesus, quebralinha, e,...",100,12,0.120000,1.895724,0.236966,...,"[ , se, na, familia, esta, jesus, quebralinha,...","[SPACE, SCONJ, ADP, NOUN, DET, ADJ, X, CCONJ, ...","[[ , SPACE], [se, SCONJ], [na, ADP], [familia,...",11,0.064327,"[[[lar, NOUN], [familia, NOUN], 0.666666666666...",1,0.010000,0,0.000000


In [227]:
list(df_analise_letra)

['artista',
 'musica',
 'letra',
 'letra_limpa',
 'letra_limpa_lista',
 'num_palavras_letra',
 'num_palavras_distintas_letra',
 'perc_palavras_distintas_letra',
 'tf_idf_sum',
 'tf_idf_medio',
 'tf_idf_medio_ajustado',
 'letra_tokens_morfossintaticos',
 'letra_elementos_morfossintaticos',
 'letra_morfossintaxe',
 'num_elementos_morfossintaticos',
 'num_elementos_morfossintaticos_relativo',
 'lista_rimas',
 'num_rimas',
 'num_rimas_relativo',
 'num_rimas_classes_diferentes',
 'num_rimas_classes_dif_relativo']

###### -> Padronizando indicadores, pela identificação do decil na distribuição

In [239]:
# Calculando decis

for coluna in [df_analise_letra['num_palavras_distintas_letra'],
               df_analise_letra['perc_palavras_distintas_letra'],
               df_analise_letra['tf_idf_sum'],
               df_analise_letra['tf_idf_medio'],
               df_analise_letra['num_elementos_morfossintaticos'],
               df_analise_letra['num_elementos_morfossintaticos_relativo'],
               df_analise_letra['num_rimas'],
               df_analise_letra['num_rimas_relativo'],
               df_analise_letra['num_rimas_classes_diferentes'],
               df_analise_letra['num_rimas_classes_dif_relativo'],
               df_analise_letra['tf_idf_medio_ajustado'],
               df_analise_letra['raridade_media_letra'],
               df_analise_letra['raridade_media_letra_ajustado']]:
    
    df_analise_letra[coluna.name+'_decil'] = coluna.apply(lambda x: ident_decil(x, coluna))
    print(coluna.name)

num_palavras_distintas_letra
perc_palavras_distintas_letra
tf_idf_sum
tf_idf_medio
num_elementos_morfossintaticos
num_elementos_morfossintaticos_relativo
num_rimas
num_rimas_relativo
num_rimas_classes_diferentes
num_rimas_classes_dif_relativo
tf_idf_medio_ajustado
raridade_media_letra
raridade_media_letra_ajustado


In [240]:
df_analise_letra.to_pickle('df_analise_letra.pkl')

###### -> Agregando indicadores por artista, pela mediana. Além disso, foi adicionado um indicador geral de vocabulário (número de termos distintos) do artista.

In [260]:
# Agregando indicadores de música por artista

df_artista_letra = df_analise_letra[['artista', 'perc_palavras_distintas_letra_decil', 'tf_idf_sum_decil', 'tf_idf_medio_decil', 'tf_idf_medio_ajustado_decil', 'raridade_media_letra_decil', 'raridade_media_letra_ajustado_decil', 'num_rimas_decil', 'num_rimas_classes_diferentes_decil']].replace('sem_info', np.nan)
df_artista_letra = df_artista_letra.groupby(['artista']).median().reset_index()
df_artista_letra['ind_rimas'] = (df_artista_letra['num_rimas_decil']+df_artista_letra['num_rimas_classes_diferentes_decil'])/2


In [261]:
lista_geral_vocabulario = []
for i in list(df_artista_letra['artista']):
    lista_geral = list(set(df_analise_letra[df_analise_letra['artista'] == i]['letra_tokens_morfossintaticos'].sum()))
    tam_lista = len(lista_geral)
    lista_geral_vocabulario.append([i, tam_lista])
df_vocabulario = pd.DataFrame(lista_geral_vocabulario)

df_artista_letra = df_artista_letra.merge(df_vocabulario, how = 'left', left_on='artista', right_on=0)
df_artista_letra['vocabulario_geral_decil'] = df_artista_letra[1].apply(lambda x: ident_decil(x, df_artista_letra[1]))
del(df_artista_letra[0])
del(df_artista_letra[1])


In [262]:
df_vocabulario.to_pickle('df_vocabulario.pkl')

###### -> Calculando nota de complexidade da letra, por artista.

In [263]:
list(df_artista_letra)[1:]

['perc_palavras_distintas_letra_decil',
 'tf_idf_sum_decil',
 'tf_idf_medio_decil',
 'tf_idf_medio_ajustado_decil',
 'raridade_media_letra_decil',
 'raridade_media_letra_ajustado_decil',
 'num_rimas_decil',
 'num_rimas_classes_diferentes_decil',
 'ind_rimas',
 'vocabulario_geral_decil']

In [264]:
df_artista_letra['nota_letra'] = (df_artista_letra['perc_palavras_distintas_letra_decil']+
                                  df_artista_letra['tf_idf_medio_decil']+
                                  df_artista_letra['ind_rimas']+
                                  df_artista_letra['vocabulario_geral_decil'])/4

df_artista_letra.sort_values('nota_letra', ascending=False).reset_index(drop=True).head(20)


,artista,perc_palavras_distintas_letra_decil,tf_idf_sum_decil,tf_idf_medio_decil,tf_idf_medio_ajustado_decil,raridade_media_letra_decil,raridade_media_letra_ajustado_decil,num_rimas_decil,num_rimas_classes_diferentes_decil,ind_rimas,vocabulario_geral_decil,nota_letra
0,djavan,9.0,7.0,7.0,9.0,8.0,9.0,4.0,4.0,4.00,10,7.5000
1,paulinho-da-viola,10.0,6.0,8.0,9.0,6.0,6.0,4.0,4.0,4.00,8,7.5000
2,tiao-carreiro-e-pardinho,8.0,9.0,2.5,6.0,7.0,8.0,9.0,9.0,9.00,10,7.3750
3,vander-lee,8.0,8.0,5.0,7.0,7.0,7.0,7.0,6.0,6.50,10,7.3750
4,xangai,8.5,7.0,6.5,7.0,10.0,9.5,7.0,7.5,7.25,7,7.3125
5,ze-geraldo,9.0,8.0,5.0,7.0,8.0,8.0,5.0,5.0,5.00,10,7.2500
6,chico-buarque,7.0,8.0,5.0,6.0,8.0,8.0,7.0,7.0,7.00,10,7.2500
7,cesar-oliveira-rogerio-melo,8.0,10.0,2.0,6.0,10.0,10.0,9.0,9.0,9.00,10,7.2500
8,oswaldo-montenegro,7.0,5.0,7.0,7.0,6.0,7.0,5.0,5.0,5.00,10,7.2500
9,nelson-goncalves,8.0,5.0,7.0,8.0,7.0,8.0,5.5,4.5,5.00,9,7.2500


In [265]:
df_artista_letra.to_pickle('df_artista_letra.pkl')

#### 2.3. Calculando nota geral do artista: complexidade harmônica + complexidade da letra

In [266]:
df_artista = pd.read_pickle('df_artista.pkl')

In [267]:
df_artista_score = df_artista.merge(df_artista_letra, how='left', on='artista')
df_artista_score['nota_final'] = (df_artista_score['nota_harmonia']*7+df_artista_score['nota_letra']*4)/11
df_artista_score.sort_values('nota_final', ascending=False).reset_index(drop=True).head(20)

,artista,num_acordes_distintos_decil,perc_acordes_fora_campo_decil,tamanho_medio_progressao_decil,media_raridade_decil,media_complexidade_decil,qtde_musicas_decil,qtde_geral_acordes_decil,nota_harmonia,perc_palavras_distintas_letra_decil,...,tf_idf_medio_decil,tf_idf_medio_ajustado_decil,raridade_media_letra_decil,raridade_media_letra_ajustado_decil,num_rimas_decil,num_rimas_classes_diferentes_decil,ind_rimas,vocabulario_geral_decil,nota_letra,nota_final
0,chico-buarque,10.0,9.0,8.0,10.0,10.0,10,10,9.571429,7.0,...,5.0,6.0,8.0,8.0,7.0,7.0,7.0,10.0,7.250,8.727273
1,djavan,9.0,9.0,8.0,10.0,10.0,10,10,9.428571,9.0,...,7.0,9.0,8.0,9.0,4.0,4.0,4.0,10.0,7.500,8.727273
2,joao-bosco,10.0,9.0,8.0,10.0,10.0,9,10,9.428571,9.0,...,5.5,8.0,9.0,9.0,5.0,4.0,4.5,10.0,7.250,8.636364
3,ivan-lins,10.0,9.0,7.0,10.0,10.0,10,10,9.428571,7.0,...,7.0,7.0,8.0,8.0,5.0,4.0,4.5,9.0,6.875,8.500000
4,tom-jobim,10.0,9.0,8.0,10.0,10.0,9,10,9.428571,7.0,...,7.0,7.0,4.0,5.0,3.0,3.0,3.0,9.0,6.500,8.363636
5,joao-gilberto,10.0,9.0,8.0,10.0,10.0,9,10,9.428571,6.0,...,8.0,7.0,6.0,6.0,4.0,4.0,4.0,8.0,6.500,8.363636
6,jorge-aragao,9.0,9.0,8.0,8.5,9.0,9,10,8.928571,8.0,...,7.0,8.0,5.5,6.0,5.0,4.0,4.5,9.0,7.125,8.272727
7,emilio-santiago,10.0,9.0,8.0,10.0,10.0,9,10,9.428571,7.0,...,7.0,8.0,5.0,5.0,4.0,4.0,4.0,7.0,6.250,8.272727
8,toquinho,10.0,9.0,7.0,9.0,9.0,10,10,9.142857,8.0,...,6.0,7.0,6.0,7.0,4.0,4.0,4.0,9.0,6.750,8.272727
9,jorge-vercillo,9.0,8.0,8.0,10.0,10.0,10,10,9.285714,6.0,...,4.0,5.0,7.0,7.0,6.0,6.0,6.0,10.0,6.500,8.272727


In [268]:
df_artista_score.to_pickle('df_artista_score.pkl')

#### 2.4. Agregando notas por estilo (ritmo)

In [269]:
df_qtde_cifras = pd.read_pickle('df_qtde_cifras.pkl')

In [270]:
df_estilos = pd.read_pickle('df_estilos.pkl')

In [271]:
df_estilos_score = df_artista_score[['artista', 'nota_harmonia', 'nota_letra', 'nota_final']]
df_estilos_score = df_estilos_score.merge(df_estilos, how = 'inner', on = 'artista')

In [272]:
df_estilos_score.to_pickle('df_estilos_score.pkl')

In [273]:
# Agregando pela média ponderada das notas dos artistas (ponderação feita pela quantidade de músicas do artista)

df_estilos_score_tmp = df_estilos_score.merge(df_qtde_cifras, left_on = 'artista', right_on = 0, how = 'inner')
df_estilos_score_tmp['fator_peso_nh'] = df_estilos_score_tmp['nota_harmonia']*df_estilos_score_tmp[1]
df_estilos_score_tmp['fator_peso_nl'] = df_estilos_score_tmp['nota_letra']*df_estilos_score_tmp[1]
df_estilos_score_tmp['fator_peso_nf'] = df_estilos_score_tmp['nota_final']*df_estilos_score_tmp[1]
df_estilos_score_grupo = df_estilos_score_tmp[['estilo', 'fator_peso_nh', 'fator_peso_nl', 'fator_peso_nf', 1]].groupby(['estilo']).sum().reset_index()

df_estilos_score_grupo['media_ponderada_nota_harmonia'] = df_estilos_score_grupo['fator_peso_nh']/df_estilos_score_grupo[1]
df_estilos_score_grupo['media_ponderada_nota_letra'] = df_estilos_score_grupo['fator_peso_nl']/df_estilos_score_grupo[1]
df_estilos_score_grupo['media_ponderada_nota_final'] = df_estilos_score_grupo['fator_peso_nf']/df_estilos_score_grupo[1]

del(df_estilos_score_grupo['fator_peso_nh'])
del(df_estilos_score_grupo['fator_peso_nl'])
del(df_estilos_score_grupo['fator_peso_nf'])
del(df_estilos_score_grupo[1])

df_estilos_score_grupo.sort_values('media_ponderada_nota_final', ascending=False)



,estilo,media_ponderada_nota_harmonia,media_ponderada_nota_letra,media_ponderada_nota_final
1,bossa-nova,8.408605,6.099947,7.569093
11,mpb,7.624080,6.450501,7.197324
20,samba,7.674243,6.254447,7.154606
12,pagode,7.548535,6.062765,7.008255
24,velha-guarda,6.912900,5.858582,6.497083
22,soul,7.066580,5.266971,6.412177
17,regional,5.497971,6.715415,5.940678
18,rock,5.862403,5.919837,5.883288
10,jovem-guarda,6.215613,5.066284,5.797675
8,indie,5.871152,5.579493,5.765094


In [274]:
df_estilos_score_grupo.to_pickle('df_estilos_score_grupo.pkl')